In [2]:
raw_data <- read.csv("Strategic_Subject_List.csv", sep=",", header=T, strip.white = T, na.strings = c("NA","NaN","","?"))
#Converting the "." in column names to "_"
names(raw_data)<-gsub("\\.", "_", names(raw_data))
cat("The Number of Rows in Initial Data is", nrow(raw_data))

The Number of Rows in Initial Data is 398684

In [3]:
final_features<-raw_data[,c('SSL_SCORE','PREDICTOR_RAT_AGE_AT_LATEST_ARREST','PREDICTOR_RAT_VICTIM_SHOOTING_INCIDENTS',
                            'PREDICTOR_RAT_VICTIM_BATTERY_OR_ASSAULT','PREDICTOR_RAT_ARRESTS_VIOLENT_OFFENSES','PREDICTOR_RAT_GANG_AFFILIATION', 
                            'PREDICTOR_RAT_NARCOTIC_ARRESTS','PREDICTOR_RAT_TREND_IN_CRIMINAL_ACTIVITY','PREDICTOR_RAT_UUW_ARRESTS','RACE_CODE_CD',
                            'LATITUDE', 'LONGITUDE', 'COMMUNITY_AREA')]

In [4]:
# Extracting the records without missing values
complete_data <- final_features[complete.cases(final_features),]
cat("The Number of Rows in Initial Data is", nrow(complete_data))

The Number of Rows in Initial Data is 224190

In [6]:
# Selecting the unique record of LATITUDE AND LOGITUDE
library(sqldf)
required_data<-sqldf("SELECT distinct((LATITUDE||LONGITUDE)) as Joioned_LAT_LONG, LATITUDE, LONGITUDE from complete_data  group by LATITUDE,LONGITUDE")
lat_long_data<-required_data[,c('LATITUDE', 'LONGITUDE')]
cat("The Number of Rows in Data is", nrow(lat_long_data))

The Number of Rows in Data is 801

In [19]:
#Copying the Data into text file to be read by python JSON Google API crawler
write.table(lat_long_data, file = "lat_long_data.txt",sep=",",row.names=FALSE,col.names=FALSE)

In [7]:
# Crawled data from python program is in zip_code_by_lat_log.csv
# Reading the data
zip_data <- read.csv("zip_code_by_lat_log.csv", sep=",", header=T, strip.white = T, na.strings = c("NA","NaN","","?"))
#Converting the "." in column names to "_"
names(zip_data)<-gsub("\\.", "_", names(zip_data))
cat("The Number of Rows in zip_data Data is", nrow(zip_data))

The Number of Rows in zip_data Data is 801

In [8]:
# Extracting the records without missing values as for 41.66733082	-87.6054492 data not found by Google Maps API
complete_data_zip_code <- zip_data[complete.cases(zip_data),]
cat("The Number of Rows in complete_data_zip_code Data is", nrow(complete_data_zip_code))

The Number of Rows in complete_data_zip_code Data is 800

In [9]:
# Selecting only LATITUDE, LOGITUDE and ZIP_CODES
complete_data_zip_code<-sqldf("SELECT LATITUDE as Z_LATITUDE, LONGITUDE as Z_LONGITUDE, ZIP_CODE_LAT_LONG  from complete_data_zip_code")

In [10]:
# Merging the records with zip_code
SSL_DATA_WITH_ZIP<-sqldf("select s.*, z.ZIP_CODE_LAT_LONG from complete_data s LEFT OUTER JOIN complete_data_zip_code z  on (s.LATITUDE = z.Z_LATITUDE AND s.LONGITUDE = z.Z_LONGITUDE)")
cat("The Number of Rows in SSL_DATA_WITH_ZIP Data is", nrow(SSL_DATA_WITH_ZIP))

The Number of Rows in SSL_DATA_WITH_ZIP Data is 224190

In [11]:
complete_SSL_DATA_WITH_ZIP<- SSL_DATA_WITH_ZIP[complete.cases(SSL_DATA_WITH_ZIP),]
cat("The Number of Rows in Data is", nrow(complete_SSL_DATA_WITH_ZIP))

The Number of Rows in Data is 223674

In [14]:
library(MatchIt)
library(ggplot2)
library(dplyr)
# Selecting only the required RACE records
Data_With_Required_Races<-sqldf("SELECT * FROM complete_SSL_DATA_WITH_ZIP where RACE_CODE_CD IN ('WHI', 'BLK', 'WBH','WWH')")
cat("The Number of Rows for analysis for algorithmic bias based on zip code(based latitudes and longitudes) =", nrow(Data_With_Required_Races))

The Number of Rows for analysis for algorithmic bias based on zip code(based latitudes and longitudes) = 220949

In [15]:
#Converting the RACE_CODE_CD into factor for Data Manipulation
Data_With_Required_Races$RACE_CODE_CD<-as.character(Data_With_Required_Races$RACE_CODE_CD)
#Changing the RACE categorical variables from BLK, WHI, WBH, WWH to 0,1,2,and 2 respectively
Data_With_Required_Races$RACE_CODE_CD[Data_With_Required_Races$RACE_CODE_CD == "BLK"]<-"0"
Data_With_Required_Races$RACE_CODE_CD[Data_With_Required_Races$RACE_CODE_CD == "WHI"]<-"1"
Data_With_Required_Races$RACE_CODE_CD[Data_With_Required_Races$RACE_CODE_CD == "WBH"]<-"2"
Data_With_Required_Races$RACE_CODE_CD[Data_With_Required_Races$RACE_CODE_CD == "WWH"]<-"2"
#Coverting back RACE_CODE_CD to factor
Data_With_Required_Races$RACE_CODE_CD<-factor(Data_With_Required_Races$RACE_CODE_CD)
#Sorting the data by zip code
Zip_Soreted_order<-sqldf("SELECT * FROM Data_With_Required_Races ORDER BY ZIP_CODE_LAT_LONG, RACE_CODE_CD")
#Using dense rank to identify the ZIP_CODE_LAT_LONG change
Zip_Soreted_order$ZIP_RANK <-dense_rank(Zip_Soreted_order$ZIP_CODE_LAT_LONG)
#Converting the required variables into factors
Zip_Soreted_order$PREDICTOR_RAT_AGE_AT_LATEST_ARREST<-factor(Zip_Soreted_order$PREDICTOR_RAT_AGE_AT_LATEST_ARREST)
Zip_Soreted_order$PREDICTOR_RAT_GANG_AFFILIATION<-factor(Zip_Soreted_order$PREDICTOR_RAT_GANG_AFFILIATION)

In [ ]:
#Creating empty Data Frame to store final results
final <- data.frame(Zip_Code= character(), Count_Of_Blacks=character(),Count_of_whites=character(),Count_of_hispanics=character(),
                        #BLACK-WHITE PSM BEFORE AND AFTER
                        BEFORE_PSM_BLACK_WHITE_T_VALUE_WHITE= character(),BEFORE_PSM_BLACK_WHITE_P_VALUE= character(), 
                        BEFORE_PSM_BLACK_WHITE_MEAN_BLACK= character(),BEFORE_PSM_BLACK_WHITE_MEAN_WHITE = character(),
                        #
                        AFTER_PSM_BLACK_WHITE_T_VALUE = character(),AFTER_PSM_BLACK_WHITE_P_VALUE = character(), 
                        AFTER_PSM_BLACK_WHITE_MEAN_BLACK = character(),AFTER_PSM_BLACK_WHITE_MEAN_WHITE = character(),
                        #AFTER_PSM_BLACK_WHITE_MEAN_DIFFERENCE=character(),
                        #BLACK-HISPANICS PSM BEFORE AND AFTER
                        BEFORE_PSM_BLACK_HISPANICS_T_VALUE= character(),BEFORE_PSM_BLACK_HISPANICS_P_VALUE= character(), 
                        BEFORE_PSM_BLACK_HISPANICS_MEAN_BLACK= character(),BEFORE_PSM_BLACK_HISPANICS_MEAN_HISPANICS = character(),
                        #
                        AFTER_PSM_BLACK_HISPANICS_T_VALUE = character(),AFTER_PSM_BLACK_HISPANICS_P_VALUE = character(), 
                        AFTER_PSM_BLACK_HISPANICS_MEAN_BLACK = character(),AFTER_PSM_BLACK_HISPANICS_MEAN_HISPANICS = character(),   
                        #AFTER_PSM_BLACK_HISPANICS_MEAN_DIFFERENCE=character(),
                        #WHITE-HISPANICS PSM BEFORE AND AFTER)
                        BEFORE_PSM_WHITE_HISPANICS_T_VALUE= character(),BEFORE_PSM_WHITE_HISPANICS_P_VALUE= character(), 
                        BEFORE_PSM_WHITE_HISPANICS_MEAN_WHITE= character(),BEFORE_PSM_WHITE_HISPANICS_MEAN_HISPANICS = character(),
                        #
                        AFTER_PSM_WHITE_HISPANICS_T_VALUE = character(),AFTER_PSM_WHITE_HISPANICS_P_VALUE = character(), 
                        AFTER_PSM_WHITE_HISPANICS_MEAN_WHITE = character(),AFTER_PSM_WHITE_HISPANICS_MEAN_HISPANICS = character())
                        #AFTER_PSM_WHITE_HISPANICS_MEAN_DIFFERENCE=character())

# Initializing the rank variable to 1
ZIP_RANK_VAR <-1
while (ZIP_RANK_VAR!=61) 
{
  
  
  sql <- sprintf("SELECT * FROM Zip_Soreted_order WHERE ZIP_RANK = %s", ZIP_RANK_VAR)
  sql_MAX<- sprintf("SELECT MAX(RACE_CODE_CD) FROM Zip_Soreted_order WHERE ZIP_RANK = %s", ZIP_RANK_VAR)
  sql_MIN<- sprintf("SELECT MIN(RACE_CODE_CD) FROM Zip_Soreted_order WHERE ZIP_RANK = %s", ZIP_RANK_VAR)
  sql_ZIP<- sprintf("SELECT DISTINCT(ZIP_CODE_LAT_LONG) FROM Zip_Soreted_order WHERE ZIP_RANK = %s", ZIP_RANK_VAR)
  sql_no_black<-sprintf("SELECT count(RACE_CODE_CD) FROM Zip_Soreted_order WHERE ZIP_RANK = %s AND RACE_CODE_CD = '0'", ZIP_RANK_VAR)
  sql_no_white<-sprintf("SELECT count(RACE_CODE_CD) FROM Zip_Soreted_order WHERE ZIP_RANK = %s AND RACE_CODE_CD = '1'", ZIP_RANK_VAR)
  sql_no_hispanics<-sprintf("SELECT count(RACE_CODE_CD) FROM Zip_Soreted_order WHERE ZIP_RANK = %s AND RACE_CODE_CD = '2'", ZIP_RANK_VAR)
  
  sql_result<-sqldf(sql)
  sql_1<-sqldf(sql_MAX)
  sql_2<-sqldf(sql_MIN)
  SQL_zip_code<-sqldf(sql_ZIP)
  sql_num_black<-sqldf(sql_no_black)
  sql_num_white<-sqldf(sql_no_white)
  sql_num_hispanics<-sqldf(sql_no_hispanics)
  no_rows<-nrow(sql_result)
  
  file_data <- data.frame(Zip_Code= character(1), Count_Of_Blacks=character(1),Count_of_whites=character(1),Count_of_hispanics=character(1),
                          #BLACK-WHITE PSM BEFORE AND AFTER
                          BEFORE_PSM_BLACK_WHITE_T_VALUE_WHITE= character(1),BEFORE_PSM_BLACK_WHITE_P_VALUE= character(1), 
                          BEFORE_PSM_BLACK_WHITE_MEAN_BLACK= character(1),BEFORE_PSM_BLACK_WHITE_MEAN_WHITE = character(1),
                          #
                          AFTER_PSM_BLACK_WHITE_T_VALUE = character(1),AFTER_PSM_BLACK_WHITE_P_VALUE = character(1), 
                          AFTER_PSM_BLACK_WHITE_MEAN_BLACK = character(1),AFTER_PSM_BLACK_WHITE_MEAN_WHITE = character(1),
                          #AFTER_PSM_BLACK_WHITE_MEAN_DIFFERENCE=character(1),
                          #BLACK-HISPANICS PSM BEFORE AND AFTER
                          BEFORE_PSM_BLACK_HISPANICS_T_VALUE= character(1),BEFORE_PSM_BLACK_HISPANICS_P_VALUE= character(1), 
                          BEFORE_PSM_BLACK_HISPANICS_MEAN_BLACK= character(1),BEFORE_PSM_BLACK_HISPANICS_MEAN_HISPANICS = character(1),
                          #
                          AFTER_PSM_BLACK_HISPANICS_T_VALUE = character(1),AFTER_PSM_BLACK_HISPANICS_P_VALUE = character(1), 
                          AFTER_PSM_BLACK_HISPANICS_MEAN_BLACK = character(1),AFTER_PSM_BLACK_HISPANICS_MEAN_HISPANICS = character(1),   
                          #AFTER_PSM_BLACK_HISPANICS_MEAN_DIFFERENCE=character(1),
                          #WHITE-HISPANICS PSM BEFORE AND AFTER)
                          BEFORE_PSM_WHITE_HISPANICS_T_VALUE= character(1),BEFORE_PSM_WHITE_HISPANICS_P_VALUE= character(1), 
                          BEFORE_PSM_WHITE_HISPANICS_MEAN_WHITE= character(1),BEFORE_PSM_WHITE_HISPANICS_MEAN_HISPANICS = character(1),
                          #
                          AFTER_PSM_WHITE_HISPANICS_T_VALUE = character(1),AFTER_PSM_WHITE_HISPANICS_P_VALUE = character(1), 
                          AFTER_PSM_WHITE_HISPANICS_MEAN_WHITE = character(1),AFTER_PSM_WHITE_HISPANICS_MEAN_HISPANICS = character(1),stringsAsFactors=FALSE) 
                          #AFTER_PSM_WHITE_HISPANICS_MEAN_DIFFERENCE=character(1))
  test<-0
  if ((no_rows == 1) || (sql_1 == sql_2 ))
  {
    file_data$Zip_Code<-SQL_zip_code[1,]
    file_data$Count_Of_Blacks<-sql_num_black[1,]
    file_data$Count_of_whites<-sql_num_white[1,]
    file_data$Count_of_hispanics<-sql_num_hispanics[1,]
    ZIP_RANK_VAR<- ZIP_RANK_VAR + 1  
    final<-rbind(final,file_data)
    test<-2
    next
  }
  else
    {
      file_data$Zip_Code<-SQL_zip_code[1,]
      file_data$Count_Of_Blacks<-sql_num_black[1,]
      file_data$Count_of_whites<-sql_num_white[1,]
      file_data$Count_of_hispanics<-sql_num_hispanics[1,]
      if (sql_num_black > 1 && sql_num_white > 1)
  {
    test<-3
    sql_BLACK_WHITE <- sprintf("SELECT * FROM Zip_Soreted_order WHERE (ZIP_RANK = %s) AND (RACE_CODE_CD = '0' OR RACE_CODE_CD = '1')", ZIP_RANK_VAR)
    SQL_REC_BLK_WHITE<-sqldf(sql_BLACK_WHITE)
    # T-test without PSM for SSL_SCORE and RACE_CODE_CD
    attach(SQL_REC_BLK_WHITE)
    t_test_results<-t.test(SSL_SCORE~RACE_CODE_CD)
    file_data$BEFORE_PSM_BLACK_WHITE_T_VALUE_WHITE<-t_test_results$statistic
    file_data$BEFORE_PSM_BLACK_WHITE_P_VALUE<-t_test_results$p.value
    file_data$BEFORE_PSM_BLACK_WHITE_MEAN_BLACK<-t_test_results$estimate[1]
    file_data$BEFORE_PSM_BLACK_WHITE_MEAN_WHITE<-t_test_results$estimate[2]
    #Count of Black and whites and hispanics
    file_data$Count_Of_Blacks<-sql_num_black[1,]
    file_data$Count_of_whites<-sql_num_white[1,]
    # PSM 
    gang_aff_check_max <- sqldf("SELECT MAX(PREDICTOR_RAT_GANG_AFFILIATION) FROM SQL_REC_BLK_WHITE ")
    gang_aff_check_min <- sqldf("SELECT MIN(PREDICTOR_RAT_GANG_AFFILIATION) FROM SQL_REC_BLK_WHITE ")
    if (gang_aff_check_max == gang_aff_check_min)
    {
            mod_match <- matchit(RACE_CODE_CD ~ PREDICTOR_RAT_AGE_AT_LATEST_ARREST+PREDICTOR_RAT_VICTIM_SHOOTING_INCIDENTS + PREDICTOR_RAT_VICTIM_BATTERY_OR_ASSAULT + 
                        PREDICTOR_RAT_ARRESTS_VIOLENT_OFFENSES + PREDICTOR_RAT_NARCOTIC_ARRESTS + PREDICTOR_RAT_TREND_IN_CRIMINAL_ACTIVITY +
                        PREDICTOR_RAT_UUW_ARRESTS,
                        method = "nearest", data = SQL_REC_BLK_WHITE)
    }
    else 
        {
            mod_match <- matchit(RACE_CODE_CD ~ PREDICTOR_RAT_AGE_AT_LATEST_ARREST+PREDICTOR_RAT_VICTIM_SHOOTING_INCIDENTS + PREDICTOR_RAT_VICTIM_BATTERY_OR_ASSAULT + 
                        PREDICTOR_RAT_ARRESTS_VIOLENT_OFFENSES + PREDICTOR_RAT_GANG_AFFILIATION + PREDICTOR_RAT_NARCOTIC_ARRESTS + PREDICTOR_RAT_TREND_IN_CRIMINAL_ACTIVITY +
                        PREDICTOR_RAT_UUW_ARRESTS,
                        method = "nearest", data = SQL_REC_BLK_WHITE)
        
    }
    #print(sql_result)
    
    matched <- match.data(mod_match)
    attach(matched)

    t_test_results<-t.test(SSL_SCORE~RACE_CODE_CD)
    file_data$Zip_Code<-SQL_zip_code[1,]
    file_data$AFTER_PSM_BLACK_WHITE_T_VALUE<-t_test_results$statistic
    file_data$AFTER_PSM_BLACK_WHITE_P_VALUE<-t_test_results$p.value
    file_data$AFTER_PSM_BLACK_WHITE_MEAN_BLACK<-t_test_results$estimate[1]
    file_data$AFTER_PSM_BLACK_WHITE_MEAN_WHITE<-t_test_results$estimate[2]

    
  }
  if (sql_num_black > 1 && sql_num_hispanics > 1)
  {
    test<-4
    sql_BLACK_hispanics <- sprintf("SELECT * FROM Zip_Soreted_order WHERE (ZIP_RANK = %s) AND (RACE_CODE_CD = '0' OR RACE_CODE_CD = '2')", ZIP_RANK_VAR)
    SQL_REC_BLK_hispanics<-sqldf(sql_BLACK_hispanics)
    SQL_REC_BLK_hispanics$RACE_CODE_CD[SQL_REC_BLK_hispanics$RACE_CODE_CD == "2"]<-"1"
    
    # T-test without PSM for SSL_SCORE and RACE_CODE_CD
    attach(SQL_REC_BLK_hispanics)
    t_test_results<-t.test(SSL_SCORE~RACE_CODE_CD)
    file_data$BEFORE_PSM_BLACK_HISPANICS_T_VALUE<-t_test_results$statistic
    file_data$BEFORE_PSM_BLACK_HISPANICS_P_VALUE<-t_test_results$p.value
    file_data$BEFORE_PSM_BLACK_HISPANICS_MEAN_BLACK<-t_test_results$estimate[1]
    file_data$BEFORE_PSM_BLACK_HISPANICS_MEAN_HISPANICS<-t_test_results$estimate[2]
    #Count of Black and hispanics
    file_data$Count_Of_Blacks<-sql_num_black[1,]
    file_data$Count_of_hispanics<-sql_num_hispanics[1,]
    # PSM 
      
    gang_aff_check_max <- sqldf("SELECT MAX(PREDICTOR_RAT_GANG_AFFILIATION) FROM SQL_REC_BLK_hispanics ")
    gang_aff_check_min <- sqldf("SELECT MIN(PREDICTOR_RAT_GANG_AFFILIATION) FROM SQL_REC_BLK_hispanics ")
    if (gang_aff_check_max == gang_aff_check_min)
    {
            mod_match <- matchit(RACE_CODE_CD ~ PREDICTOR_RAT_AGE_AT_LATEST_ARREST+PREDICTOR_RAT_VICTIM_SHOOTING_INCIDENTS + PREDICTOR_RAT_VICTIM_BATTERY_OR_ASSAULT + 
                        PREDICTOR_RAT_ARRESTS_VIOLENT_OFFENSES + PREDICTOR_RAT_NARCOTIC_ARRESTS + PREDICTOR_RAT_TREND_IN_CRIMINAL_ACTIVITY +
                        PREDICTOR_RAT_UUW_ARRESTS,
                        method = "nearest", data = SQL_REC_BLK_hispanics)
    }
    else 
        {
            mod_match <- matchit(RACE_CODE_CD ~ PREDICTOR_RAT_AGE_AT_LATEST_ARREST+PREDICTOR_RAT_VICTIM_SHOOTING_INCIDENTS + PREDICTOR_RAT_VICTIM_BATTERY_OR_ASSAULT + 
                        PREDICTOR_RAT_ARRESTS_VIOLENT_OFFENSES + PREDICTOR_RAT_GANG_AFFILIATION + PREDICTOR_RAT_NARCOTIC_ARRESTS + PREDICTOR_RAT_TREND_IN_CRIMINAL_ACTIVITY +
                        PREDICTOR_RAT_UUW_ARRESTS,
                        method = "nearest", data = SQL_REC_BLK_hispanics)
        
    }
    #print(sql_result)
    
    matched <- match.data(mod_match)
    attach(matched)

    t_test_results<-t.test(SSL_SCORE~RACE_CODE_CD)
    file_data$Zip_Code<-SQL_zip_code[1,]
    file_data$AFTER_PSM_BLACK_HISPANICS_T_VALUE<-t_test_results$statistic
    file_data$AFTER_PSM_BLACK_HISPANICS_P_VALUE<-t_test_results$p.value
    file_data$AFTER_PSM_BLACK_HISPANICS_MEAN_BLACK<-t_test_results$estimate[1]
    file_data$AFTER_PSM_BLACK_HISPANICS_MEAN_HISPANICS<-t_test_results$estimate[2]
    
  }
  if (sql_num_white > 1 && sql_num_hispanics > 1)
  {
    test<-5
    sql_white_hispanics <- sprintf("SELECT * FROM Zip_Soreted_order WHERE (ZIP_RANK = %s) AND (RACE_CODE_CD = '1' OR RACE_CODE_CD = '2')", ZIP_RANK_VAR)
    SQL_REC_white_hispanics<-sqldf(sql_white_hispanics)
    SQL_REC_white_hispanics$RACE_CODE_CD[SQL_REC_white_hispanics$RACE_CODE_CD == "1"]<-"0"
    SQL_REC_white_hispanics$RACE_CODE_CD[SQL_REC_white_hispanics$RACE_CODE_CD == "2"]<-"1"
    
    # T-test without PSM for SSL_SCORE and RACE_CODE_CD
    attach(SQL_REC_white_hispanics)
    t_test_results<-t.test(SSL_SCORE~RACE_CODE_CD)
    file_data$BEFORE_PSM_WHITE_HISPANICS_T_VALUE<-t_test_results$statistic
    file_data$BEFORE_PSM_WHITE_HISPANICS_P_VALUE<-t_test_results$p.value
    file_data$BEFORE_PSM_WHITE_HISPANICS_MEAN_WHITE<-t_test_results$estimate[1]
    file_data$BEFORE_PSM_WHITE_HISPANICS_MEAN_HISPANICS<-t_test_results$estimate[2]
    #Count of Black and hispanics
    file_data$Count_of_whites<-sql_num_white[1,]
    file_data$Count_of_hispanics<-sql_num_hispanics[1,]
    # PSM 
    gang_aff_check_max <- sqldf("SELECT MAX(PREDICTOR_RAT_GANG_AFFILIATION) FROM SQL_REC_white_hispanics ")
    gang_aff_check_min <- sqldf("SELECT MIN(PREDICTOR_RAT_GANG_AFFILIATION) FROM SQL_REC_white_hispanics ")
    if (gang_aff_check_max == gang_aff_check_min)
    {
            mod_match <- matchit(RACE_CODE_CD ~ PREDICTOR_RAT_AGE_AT_LATEST_ARREST+PREDICTOR_RAT_VICTIM_SHOOTING_INCIDENTS + PREDICTOR_RAT_VICTIM_BATTERY_OR_ASSAULT + 
                        PREDICTOR_RAT_ARRESTS_VIOLENT_OFFENSES + PREDICTOR_RAT_NARCOTIC_ARRESTS + PREDICTOR_RAT_TREND_IN_CRIMINAL_ACTIVITY +
                        PREDICTOR_RAT_UUW_ARRESTS,
                        method = "nearest", data = SQL_REC_white_hispanics)
    }
    else 
        {
            mod_match <- matchit(RACE_CODE_CD ~ PREDICTOR_RAT_AGE_AT_LATEST_ARREST+PREDICTOR_RAT_VICTIM_SHOOTING_INCIDENTS + PREDICTOR_RAT_VICTIM_BATTERY_OR_ASSAULT + 
                        PREDICTOR_RAT_ARRESTS_VIOLENT_OFFENSES + PREDICTOR_RAT_GANG_AFFILIATION + PREDICTOR_RAT_NARCOTIC_ARRESTS + PREDICTOR_RAT_TREND_IN_CRIMINAL_ACTIVITY +
                        PREDICTOR_RAT_UUW_ARRESTS,
                        method = "nearest", data = SQL_REC_white_hispanics)
        
    }

    
   
    matched <- match.data(mod_match)
    attach(matched)

    t_test_results<-t.test(SSL_SCORE~RACE_CODE_CD)
    file_data$Zip_Code<-SQL_zip_code[1,]
    file_data$AFTER_PSM_WHITE_HISPANICS_T_VALUE<-t_test_results$statistic
    file_data$AFTER_PSM_WHITE_HISPANICS_P_VALUE<-t_test_results$p.value
    file_data$AFTER_PSM_WHITE_HISPANICS_MEAN_WHITE<-t_test_results$estimate[1]
    file_data$AFTER_PSM_WHITE_HISPANICS_MEAN_HISPANICS<-t_test_results$estimate[2]
    
  }
    }
  
  
  ZIP_RANK_VAR<- ZIP_RANK_VAR + 1
  final<-rbind(final,file_data)
}


In [19]:
library(xlsx)
nrow(final)

Loading required package: rJava
Loading required package: xlsxjars


[1] 60

In [28]:
write.xlsx(final, file = "Algorithmic_bias_by_zip_.xlsx",row.names=FALSE)